In [1]:
!pip install tensorflow==1.15
!pip install stable-baselines[mpi]==2.10.2

In [1]:
import os
import gym
import numpy as np
import pandas as pd
from gym import spaces

There are 6 possible log levels. Here it is a list of them starting from high verbosity to low:
* 1. Trace : detail info for tracing
* 2. Debug
* 3. Info : important but normal
* 4. Warn
* 5. Error
* 6. Fatal

Among all these 6 log levels, we have decided to eliminate **Error** and **Fatal** as these two are mostly related to runtime behavioir while the information that our model try to make its logging decisions only comes from static source code features. There fore it does not have enough knowledge to decide about logs with Error and Fatal level.


**Action space prespective:**


<img src= "images/action_space_implementations.png" alt="alt text"  />

For the first run of the implementation we will consider all of the for levels (**Combined**), and after this we implement the other way to compare them together.

For the first run implementatoin the **action space** takes **5** discrete values; 
* **"0"**: not_Log
* **"1"**: Trace
* **"2"**: Debug
* **"3"**: Info
* **"4"**: Warn

**Observatio space prespective:**

Observation space is going to be a vector (box) representing each function (rows of our dataset). It has shape of (1,14) it is a vector and we have 14 features to illusterate the fuction under study.

**Reward Function:**

<img src="images/reward_function.drawio.png" />

<img src="images/sigmoid1.png" 
 longdesc="https://www.wolframalpha.com/input?i=plot+%28%28sigmoid+function+0.3x+%29+minus+0.5+%29+multiply+by+2++from+-10+to+10" title="from WolframAlpha"/>

<img src="images/sigmoid2.png" />


In [26]:
class source_code_env(gym.Env):
    metadata = {'render.modes': ['console']}
    
    number_of_actions = 5
    
    
    actions = {
        "NOT_LOG": 0,
        "TRACE": 1,
        "DEBUG": 2,
        "INFO": 3,
        "WARN": 4
    }
    
    features_for_reward_func = {
        "number_of_loops": 0,
        "nesting_level": 1,
        "input_dependant_level": 2,
        "number_of_defined_threads": 3,
        "number_of_started_threads": 4,
        "number_of_join_threads": 5,
        "number_of_defined_locks": 6,
        "number_of_acquired_locks_threads": 7,
        "number_of_released_locks": 8,
        "number_of_usage_of_extra": 9,
        "number_of_usage_of_values_list": 10,
        "number_of_usage_of_select_related": 11
    }
    
    
    def __init__(self, dataset):
        super(source_code_env, self).__init__()
        self.dataset = dataset.copy()
        self.action_space = spaces.Discrete(number_of_actions)
        self.observation_space = spaces.Box(low= dataset.min().min(), 
                                            high= dataset.max().max(), 
                                            shape=(len(dataset.columns),))
        self.t = 0
        self.t_limit = len(dataset)
        
    def _action_determiner(self, obs):
        pass
    
    def _sigmoid(self, x_coefficient):
        pass
    
    def _reward(self, obs, taken_action):
        
        """
        arguments:
        
        obs: is supposed to be a vector (a row of the pandas dataframe);
        taken action: is the action taken by model based on obs. Its value could be {0, 1 ... 4}.
        """
        """action_based_on_obs = None
        
        if(obs.sum() == 0):
            action_based_on_obs = actions("NOT_LOG")
        else:
            action_based_on_obs = actions("TRACE")"""
        
        determined_action = _action_determiner(obs)
        
        if taken_action == determined_action: # if the answer is correct
            reward = 1 + _sigmoid()
        elif determined_action == 0 and taken_action != 0: # if it was not supposed to be logged but it logged
            reward = -4 #Worse scenario 0 - 4
        elif determined_action != 0 and taken_action == 0: # if it was supposed to be logged but it did not log
            reward = -4 #Worse scenario 0 - 4
        else: # wrong log level
            reward = -abs(taken_action - determined_action) # will be -3 at the worse
        
    
    def reset(self):
        self.t = 0
        obs = self.dataset.iloc[self.t].astype(np.float32)
        return obs
    
    def step(self, action):
        
        done = False
        
        
        if self.t >= self.t_limit:
            done = True
        
        obs = self.dataset.iloc[self.t].astype(np.float32) # getting current observation to compute reward
        reward = self._reward(obs, action)
        self.t += 1
        obs = self.dataset.iloc[self.t].astype(np.float32) # getting updated observation after taken action
        
        info={}
        
        return obs, reward, done, info

# Future Work:

* Ading Function names
* Adding file names 
* TF-IDF
* Read/Write
* API MIsuses scenarios
* GP
* Trying different implementaton of reward function
    * Adding "NO_ACTION" as one of the actions
    * Mathematical implementation
    * Negative Reward
    * Normalization of the data
* Adding more source code


# Garbage

In [ ]:
class LogEnvironment(gym.Env):
    metadata = {'render.modes': ['console']}
    
    
    def __init__(self,dataset, high = None, low = None):
        super(LogEnvironment, self).__init__()
        self.rewards = []
        self.dataset = dataset
        self.states = np.array(dataset.iloc[:,:-1])
        self.state = 0
        self.predict_classes = []
        
        n_actions = 4
        self.action_space = spaces.Discrete(n_actions)
        if high is None:
            low = np.array(dataset.iloc[:,:-1].min()).astype(np.float32)
            high = np.array(dataset.iloc[:,:-1].max()).astype(np.float32)
            self.observation_space = spaces.Box(low=low, high=high, dtype=np.float32)
        else:
            self.observation_space = spaces.Box(low=low, high=high, dtype=np.float32)
        
    
    def reset(self):
        self.state = 0
        LogEnvironment.predicts.append(self.predict_classes)
        self.predict_classes = []
        return self.states[self.state].astype(np.float32)
    
    def step(self, action):
        if action == self.NoLog:
            self.predict_classes.append(0)
        elif action == self.Log:
            self.predict_classes.append(1)
        else:
            raise ValueError("Received invalid action={} which is not part of the action space".format(action))
        
        done = bool(self.state >= len(self.classes)-1)
        
        
        
        reward = 1 if self.predict_classes[self.state] == self.classes[self.state] else 0
        self.rewards.append(reward)
        
        self.state += 0 if done else 1
        
        info = {}
        
        return self.states[self.state].astype(np.float32), reward, done, info
        

In [37]:
class test:
    d = []
    p = 1
    def __init__(self, dataset):
        self.dataset = dataset.copy()
        test.d.append(p)
        
    def fun(self):
        self.dataset.append("a")

In [38]:
lst = ["Amir"]
cls = test(lst)

NameError: name 'p' is not defined

In [36]:
lst

['Amir']

In [35]:
cls.dataset

['Amir', 'a']

In [33]:
cls.d

['b']

In [6]:
features_for_reward_func = {
        "number_of_loops": 0,
        "nesting_level": 1,
        "input_dependant_level": 2,
        "number_of_defined_threads": 3,
        "number_of_started_threads": 4,
        "number_of_join_threads": 5,
        "number_of_defined_locks": 6,
        "number_of_acquired_locks_threads": 7,
        "number_of_released_locks": 8,
        "number_of_usage_of_extra": 9,
        "number_of_usage_of_values_list": 10,
        "number_of_usage_of_select_related": 11
    }
features_for_reward_func["number_of_loops"]

0